
<div dir="rtl" align="right">

## تکلیف

محاسبه‌ی پذیرفتاری مغناطیسی

تکلیف مدل آیزینگ

# افزودن میدان مغناطیسی خارجی به مدل آیزینگ

برای افزودن یک میدان مغناطیسی خارجی \( h \)، ترم جدیدی به هامیلتونی اضافه می‌شود:

$$
H = -J \sum_{\langle i,j \rangle} S_i S_j - h \sum_i S_i
$$

- $ h $: میدان مغناطیسی خارجی.
- $ \sum_i S_i $: مجموع تمامی اسپین‌ها در سیستم.

---

# **محاسبه‌ی مغناطش متوسط (Magnetization)**

مغناطش \( M \) به صورت میانگین مجموع اسپین‌ها تعریف می‌شود:

$$
M = \frac{1}{N} \sum_i S_i
$$

که در آن \( N \) تعداد اسپین‌ها در سیستم است.

---

# **محاسبه‌ی پذیرفتاری مغناطیسی (Magnetic Susceptibility)**

حساسیت مغناطیسی $\chi$ نشان‌دهنده‌ی پاسخ سیستم به تغییرات میدان مغناطیسی خارجی است و به صورت زیر تعریف می‌شود:

$$
\chi = \frac{\partial M}{\partial h}
$$

این کمیت را می‌توان به صورت نوسانات مغناطش نیز بیان کرد:

$$
\chi = \frac{1}{kT} \left( \langle M^2 \rangle - \langle M \rangle^2 \right)
$$

که در آن:
- $ k $: ثابت بولتزمان.
- $ T $: دما.
- $ \langle M \rangle $: میانگین مغناطش.
- $ \langle M^2 \rangle $: میانگین مربع مغناطش.

---

+++

- با استفاده از مقادیر مغناطش، حساسیت مغناطیسی $ \chi $ را محاسبه کنید.
- تغییرات $ \chi $ را بر حسب دما $ T $ و میدان مغناطیسی $ h $ بررسی کنید.
</div>

In [1]:
import numpy as np
from scipy.ndimage import convolve
class IsingModel(np.ndarray):
    def __new__(cls , N , *args , **kwargs ):
        obj = np.random.choice([1 , -1] , size=[N,N] ).view(cls)
        return obj

    def __array_finalize__(self , obj):
        if obj is None: return

    def __init__(self , N):
        self.N = N
        self._filter = np.array([[0, 1, 0], [1, 0, 1], [0, 1, 0]])
        self._neighbors = convolve(self , self.filter , mode='wrap')

    @property
    def magnetization(self):
        return float(np.sum(self))

    @property
    def filter(self):
        return self._filter

    @property
    def neighbors(self):
        return self._neighbors

    @property
    def energy(self):
        return float(-0.5*np.sum( self*self.neighbors ))

    def monte_carlo_step(self, temperature : float ) -> bool:
        """
            Perform a single Monte Carlo sweep with sequential spin updates.
            Returns the number of accepted spin flips.    
        """
        i = np.random.randint(0, self.N-1)
        j = np.random.randint(0, self.N-1)

        # Calculate the energy change if this spin is flipped
        delta_E = 2 * self[i, j] * self.neighbors[i, j]
        # Apply Metropolis criterion
        if delta_E <= 0 or np.random.rand() < np.exp(-delta_E / temperature):
            self[i, j] *= -1  # Flip the spin
            # Update the n_sum matrix around the (i, j) element
            self._neighbors[ i+1 % self.N , j] += 2 * self[i, j]
            self._neighbors[ i-1 , j] += 2 * self[i, j]
            self._neighbors[ i , j+1 % self.N] += 2 * self[i, j]
            self._neighbors[ i , j-1] += 2 * self[i, j]
            return True
        return False